In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from google.colab import files

# --- Завантаження файлу з локального комп’ютера ---
uploaded = files.upload()  # З’явиться вікно для вибору файлу

# Отримаємо ім'я завантаженого файлу
filename = list(uploaded.keys())[0]
print(f"Завантажено файл: {filename}")

# --- Зчитування даних ---
df = pd.read_csv(filename, encoding='latin-1')
df = df.rename(columns={'v1':'label', 'v2':'message'})
df = df[['label', 'message']]

print("Перші 5 рядків датасету:")
print(df.head())

# --- Аналіз ---
print("\nРозподіл класів:")
print(df['label'].value_counts())

# --- Підготовка міток ---
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])  # spam=1, ham=0

# --- Токенізація і паддінг ---
max_vocab = 5000
max_len = 100

tokenizer = Tokenizer(num_words=max_vocab, oov_token="<OOV>")
tokenizer.fit_on_texts(df['message'])

sequences = tokenizer.texts_to_sequences(df['message'])
padded = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

# --- Розбивка на train/test ---
X_train, X_test, y_train, y_test = train_test_split(padded, df['label_encoded'], test_size=0.2, random_state=42)

# --- Побудова моделі ---
model = Sequential([
    Embedding(max_vocab, 64, input_length=max_len),
    LSTM(64),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# --- Навчання ---
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

# --- Прогнози ---
pred_probs = model.predict(X_test)
pred_labels = (pred_probs > 0.5).astype(int).flatten()

# --- Вивід 200 прикладів ---
print("\nПерші 200 повідомлень з реальним та прогнозованим класом:")
for i in range(min(200, len(X_test))):
    print(f"Повідомлення: {tokenizer.sequences_to_texts([X_test[i]])[0]}")
    print(f"Справжній клас: {le.inverse_transform([y_test.iloc[i]])[0]}, Прогноз: {le.inverse_transform([pred_labels[i]])[0]}")
    print("-"*50)


Saving archive (1).zip to archive (1).zip
Завантажено файл: archive (1).zip
Перші 5 рядків датасету:
  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

Розподіл класів:
label
ham     4825
spam     747
Name: count, dtype: int64


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 13s 96ms/step - accuracy: 0.8572 - loss: 0.4653 - val_accuracy: 0.8655 - val_loss: 0.3961
Epoch 2/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - accuracy: 0.8612 - loss: 0.4110 - val_accuracy: 0.8655 - val_loss: 0.3988
Epoch 3/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 10s 74ms/step - accuracy: 0.8751 - loss: 0.3825 - val_accuracy: 0.8655 - val_loss: 0.4018
Epoch 4/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 10s 73ms/step - accuracy: 0.8639 - loss: 0.4038 - val_accuracy: 0.8655 - val_loss: 0.3960
Epoch 5/5
70/70 ━━━━━━━━━━━━━━━━━━━━ 11s 87ms/step - accuracy: 0.8633 - loss: 0.4058 - val_accuracy: 0.8655 - val_loss: 0.3960
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step

Перші 200 повідомлень з реальним та прогнозованим класом:
Повідомлення: funny fact nobody teaches <OOV> 2 <OOV> tsunamis 2 <OOV> <OOV> 2 <OOV> <OOV> no 1 teaches hw 2 choose a wife natural <OOV> just happens <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OO